<a href="https://colab.research.google.com/github/JosiasRuiz/AdvanceML/blob/main/A1b_DL_TC5033_AD2023_Eq43.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TC 5033
## Deep Learning
## Fully Connected Deep Neural Networks

#### Activity 1b: Implementing a Fully Connected Network for Kaggle ASL Dataset

- Objective

The aim of this part of the activity is to apply your understanding of Fully Connected Networks by implementing a multilayer network for the [Kaggle ASL (American Sign Language) dataset](https://www.kaggle.com/datasets/grassknoted/asl-alphabet). While you have been provided with a complete solution for a Fully Connected Network using Numpy for the MNIST dataset, you are encouraged to try to come up with the solution.

- Instructions

    This activity requires submission in teams of 3 or 4 members. Submissions from smaller or larger teams will not be accepted unless prior approval has been granted (only due to exceptional circumstances). While teamwork is encouraged, each member is expected to contribute individually to the assignment. The final submission should feature the best arguments and solutions from each team member. Only one person per team needs to submit the completed work, but it is imperative that the names of all team members are listed in a Markdown cell at the very beginning of the notebook (either the first or second cell). Failure to include all team member names will result in the grade being awarded solely to the individual who submitted the assignment, with zero points given to other team members (no exceptions will be made to this rule).

    Load and Preprocess Data: You are provided a starter code to load the data. Be sure to understand the code.

    Review MNIST Notebook (Optional): Before diving into this activity, you have the option to revisit the MNIST example to refresh your understanding of how to build a Fully Connected Network using Numpy.

    Start Fresh: Although you can refer to the MNIST solution at any point, try to implement the network for the ASL dataset on your own. This will reinforce your learning and understanding of the architecture and mathematics involved.

    Implement Forward and Backward Pass: Write the code to perform the forward and backward passes, keeping in mind the specific challenges and characteristics of the ASL dataset.
    
     Design the Network: Create the architecture of the Fully Connected Network tailored for the ASL dataset. Choose the number of hidden layers, neurons, and hyperparameters judiciously.

    Train the Model: Execute the training loop, ensuring to track performance metrics such as loss and accuracy.

    Analyze and Document: Use Markdown cells to document in detail the choices you made in terms of architecture and hyperparameters, you may use figures, equations, etc to aid in your explanations. Include any metrics that help justify these choices and discuss the model's performance.  

- Evaluation Criteria

    - Code Readability and Comments
    - Appropriateness of chosen architecture and hyperparameters for the ASL dataset
    - Performance of the model on the ASL dataset (at least 70% acc)
    - Quality of Markdown documentation

- Submission

Submit this Jupyter Notebook in canvas with your complete solution, ensuring your code is well-commented and includes Markdown cells that explain your design choices, results, and any challenges you encountered.


In [9]:
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import os

from get_images import get_images


%load_ext autoreload
%autoreload 2
#################################
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense
import cv2
import pandas as pd
import random
import os
import ntpath
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from imgaug import augmenters as iaa
import subprocess

np.random.seed(0)

In [3]:
!git clone https://github.com/JosiasRuiz/AdvanceML.git

Cloning into 'AdvanceML'...
remote: Enumerating objects: 84534, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 84534 (delta 1), reused 14 (delta 1), pack-reused 84520
Receiving objects: 100% (84534/84534), 1004.25 MiB | 28.25 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (84515/84515), done.


In [52]:
train_dir = 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_train'
test_dir = 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_test'
sign_classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','del','nothing','space']

In [54]:
FileList = []
DirTMP = train_dir
print(DirTMP)
list_files = subprocess.check_output('ls '+ DirTMP, shell=True).decode()
print(list_files)
list_files = list_files.split()
for File in list_files:
  print(File)
  FileLab = File.split("_")
  FileList.append([FileLab[0], DirTMP + File])
print(list_files[0:5])
print("Files in path:" + str(len(list_files)))
print(FileList[0:5])
test_df = pd.DataFrame(FileList, columns =['label','file'])

AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_train
A
B
C
D
del
E
F
G
H
I
J
K
L
M
N
nothing
O
P
Q
R
S
space
T
U
V
W
X
Y
Z

A
B
C
D
del
E
F
G
H
I
J
K
L
M
N
nothing
O
P
Q
R
S
space
T
U
V
W
X
Y
Z
['A', 'B', 'C', 'D', 'del']
Files in path:29
[['A', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_trainA'], ['B', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_trainB'], ['C', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_trainC'], ['D', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_trainD'], ['del', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_traindel']]


In [16]:
FileList = []
for sign_class in sign_classes:
  DirTMP = train_dir + "/"+sign_class+"/"
  list_files = subprocess.check_output('ls '+ DirTMP, shell=True).decode()
  list_files = list_files.split()
  for ListFile in list_files:
    FileList.append([sign_class,DirTMP+ListFile])
print(FileList[0:5])
print("Files in path:" + str(len(FileList)))

[['A', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_train/A/A561.jpg'], ['A', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_train/A/A562.jpg'], ['A', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_train/A/A563.jpg'], ['A', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_train/A/A564.jpg'], ['A', 'AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_train/A/A565.jpg']]
Files in path:84486


In [21]:
train_df=pd.DataFrame(FileList, columns =['label', 'file'])

In [ ]:
# DATA_PATH = '/media/pepe/DataUbuntu/Databases/asl_data/'
DATA_PATH = '/home/pepe/Documents/github_repos/datasets/asl_data'
train_df = pd.read_csv(os.path.join(DATA_PATH, 'sign_mnist_train.csv'))
valid_df = pd.read_csv(os.path.join(DATA_PATH, 'sign_mnist_valid.csv'))

In [36]:
train_df.head()

,label,file
0,A,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
1,A,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
2,A,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
3,A,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
4,A,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...


In [40]:
test_df.head()

,label,file
0,A,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
1,B,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
2,C,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
3,D,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...
4,E,AdvanceML/ML_Act01/ASL_Alphabet/asl_alphabet_t...


### Importar Images

In [ ]:
y_train = np.array(train_df['label'])
y_val = np.array(valid_df['label'])
del train_df['label']
del valid_df['label']
x_train = train_df.values.astype(np.float32)
x_val = valid_df.values.astype(np.float32)

In [ ]:
def split_val_test(x, y, pct=0.5, shuffle=True):
    '''
    Create a function that will allow you to split the previously loaded validation set
    into valition and test.
    '''
    pass

In [ ]:
x_val, y_val, x_test, y_test = split_val_test(x_val, y_val)

In [ ]:
### The following

alphabet=list(string.ascii_lowercase)
alphabet.remove('j')
alphabet.remove('z')
print(len(alphabet))

### Normalise

### Graficar muestras

### Ecuaciones para nuestro modelo


$$z^1 = W^1 X + b^1$$

$$a^1 = ReLU(z^1) $$

$$z^2 = W^2 a^1 + b^2$$

$$\hat{y} = \frac{e^{z^{2_k}}}{\sum_j{e^{z_j}}}$$


$$ \mathcal{L}(\hat{y}^{i}, y^{i}) =  - y^{i}  \ln(\hat{y}^{i}) = -\ln(\hat{y}^i)$$


$$ \mathcal{J}(w, b) =  \frac{1}{num\_samples} \sum_{i=1}^{num\_samples}-\ln(\hat{y}^{i})$$

### Funciones adicionales

#### Mini batches

## Nuestra clase Linear, ReLU y Sequential

###  Clase Linear

### Clase ReLU

### Clase Sequential

### Cost Function

### Loop de entrenamiento

### Create your model and train it

### Test your model on Random data from your test set

In [ ]:
idx = np.random.randint(len(y_test))
plot_number(x_test[idx].reshape(28,28))
pred = model.predict(x_test[idx].reshape(-1, 1))
print(f'el valor predicho es: {alphabet[pred]} el valor real es:{alphabet[y_test[idx]]}')